# Hyperparameter Optimization of Decision Tree Classifier Using Hill Climbing and Genetic Algorithms

## Introduction

In this notebook, we focus on hyperparameter optimization for a **Decision Tree Classifier** using **Hill Climbing** and **Genetic Algorithms**. Decision trees are simple yet powerful models capable of capturing complex decision boundaries. However, they are prone to overfitting, making hyperparameter tuning crucial for optimal performance.

Our goal is to fine-tune the hyperparameters to maximize the classifier's accuracy on the **Breast Cancer Wisconsin** dataset. By applying advanced optimization techniques, we aim to systematically explore the hyperparameter space and enhance the model's predictive capabilities.

## Dataset

We utilize the **Breast Cancer Wisconsin dataset**, consisting of **569 samples** with **30 numerical features**. The dataset provides a binary classification task to distinguish between malignant and benign tumors based on various measurements.

## Hyperparameter Space

We define a diverse hyperparameter space for the Decision Tree classifier:

- **`criterion`**: Function to measure the quality of a split (`"gini"` or `"entropy"`).
- **`splitter`**: Strategy used to choose the split at each node (`"best"` or `"random"`).
- **`max_depth`**: Maximum depth of the tree. Values range from `None` (unlimited depth) to integers from 1 to 20.
- **`min_samples_split`**: Minimum number of samples required to split an internal node (integer between 2 and 20).
- **`min_samples_leaf`**: Minimum number of samples required to be at a leaf node (integer between 1 and 20).
- **`max_features`**: Number of features to consider when looking for the best split (`None`, `"auto"`, `"sqrt"`, `"log2"`).

## Optimization Methods

### Hill Climbing Optimizer

- **Description**: Iteratively explores neighboring configurations from a starting point, seeking improvements.
- **Strengths**: Efficient for local search and requires less computational resources.
- **Challenges**: Susceptible to becoming trapped in local optima.

### Genetic Algorithm Optimizer

- **Description**: Utilizes mechanisms inspired by biological evolution, such as selection, crossover, and mutation, to evolve a population of solutions.
- **Strengths**: Capable of exploring a broad search space and avoiding premature convergence.
- **Challenges**: Requires careful parameter tuning (e.g., population size, mutation rate) and more computational resources.

## Evaluation Function

Our evaluation function:

- Creates a Decision Tree classifier using hyperparameters passed via `**config`.
- Implements **Stratified K-Fold Cross-Validation** to ensure reliable performance estimates.
- Employs fidelity levels corresponding to cross-validation folds, enabling a trade-off between evaluation cost and fidelity.

## Goals

- **Optimize Hyperparameters**: Discover the hyperparameter set that maximizes validation accuracy.
- **Evaluate Optimization Techniques**: Assess which optimization strategy is more effective for this problem.
- **Efficient Budget Utilization**: Complete the optimization within a specified computational budget.

## Expected Outcomes

- Enhanced performance of the Decision Tree classifier through optimized hyperparameters.
- Understanding of which hyperparameters most significantly influence model performance.
- Performance comparison between Hill Climbing and Genetic Algorithms in the context of hyperparameter optimization.

In [ ]:
cd ..

In [ ]:
import logging
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from typing import Dict, Any
import numpy as np

# Importing necessary classes from your module
from focus_opt.hp_space import HyperParameterSpace, CategoricalHyperParameter, OrdinalHyperParameter, ContinuousHyperParameter
from focus_opt.optimizers import HillClimbingOptimizer, GeneticAlgorithmOptimizer

# Set up logging
logging.basicConfig(level=logging.INFO)

# Define the hyperparameter space for a Decision Tree Classifier
hp_space = HyperParameterSpace("Decision Tree HP Space")

hp_space.add_hp(CategoricalHyperParameter(name="criterion", values=["gini", "entropy"]))
hp_space.add_hp(CategoricalHyperParameter(name="splitter", values=["best", "random"]))
hp_space.add_hp(OrdinalHyperParameter(name="max_depth", values=[None] + list(range(1, 21))))
hp_space.add_hp(ContinuousHyperParameter(name="min_samples_split", min_value=2, max_value=20, is_int=True))
hp_space.add_hp(ContinuousHyperParameter(name="min_samples_leaf", min_value=1, max_value=20, is_int=True))
hp_space.add_hp(ContinuousHyperParameter(name="max_features", min_value=0.0, max_value=1.0, step_size=0.05))

# Load the Breast Cancer dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Refactored evaluation function for Decision Tree Classifier
def dt_evaluation(config: Dict[str, Any], fidelity: int) -> float:
    """
    Evaluation function for a Decision Tree Classifier with cross-validation.

    Args:
        config (Dict[str, Any]): Configuration of hyperparameters.
        fidelity (int): Fidelity level (index of the CV fold).

    Returns:
        float: Accuracy for the specified CV fold.
    """
    logging.info(f"Evaluating config: {config} at fidelity level: {fidelity}")

    # Initialize the Decision Tree Classifier with the config as **kwargs
    # Set a fixed random_state for reproducibility
    clf = DecisionTreeClassifier(random_state=42, **config)

    # Perform cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Get the train and test indices for the specified fold
    for fold_index, (train_index, test_index) in enumerate(skf.split(X, y)):
        if fold_index + 1 == fidelity:
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            clf.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            logging.info(f"Score for config {config} at fold {fidelity}: {score}")
            return score

    raise ValueError(f"Invalid fidelity level: {fidelity}")

# Instantiate the HillClimbingOptimizer
hill_climbing_optimizer = HillClimbingOptimizer(
    hp_space=hp_space,
    evaluation_function=dt_evaluation,
    max_fidelity=5,
    maximize=True,
    log_results=True,
    warm_start=20,
    random_restarts=5,
)

# Run the Hill Climbing optimization
best_candidate_hill_climbing = hill_climbing_optimizer.optimize(budget=500)
print(f"Best candidate from Hill Climbing: {best_candidate_hill_climbing.config} with score: {best_candidate_hill_climbing.evaluation_score}")

# Instantiate the GeneticAlgorithmOptimizer
ga_optimizer = GeneticAlgorithmOptimizer(
    hp_space=hp_space,
    evaluation_function=dt_evaluation,
    max_fidelity=5,
    maximize=True,
    population_size=20,
    crossover_rate=0.8,
    mutation_rate=0.1,
    elitism=1,
    tournament_size=3,
    min_population_size=5,
    log_results=True
)

# Run the Genetic Algorithm optimization
best_candidate_ga = ga_optimizer.optimize(budget=500)
print(f"Best candidate from Genetic Algorithm: {best_candidate_ga.config} with score: {best_candidate_ga.evaluation_score}")


In [ ]:
print(f"Best candidate from Hill Climbing: {best_candidate_hill_climbing.config} with score: {best_candidate_hill_climbing.evaluation_score}")

In [ ]:
print(f"Best candidate from Genetic Algorithm: {best_candidate_ga.config} with score: {best_candidate_ga.evaluation_score}")
